# HW3 Image Classification
**因需跑超過colab限制時間，因此將此份code轉成py檔，放到cluster上跑。**
## We strongly recommend that you run with Kaggle for this homework
https://www.kaggle.com/c/ml2022spring-hw3b/code?competitionId=34954&sortBy=dateCreated

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.

In [1]:
#! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip
! wget -O food11.zip "https://github.com/virginiakm1988/ML2022-Spring/blob/main/HW03/food11.zip?raw=true"

--2022-03-24 08:14:32--  https://github.com/virginiakm1988/ML2022-Spring/blob/main/HW03/food11.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW03/food11.zip [following]
--2022-03-24 08:14:32--  https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW03/food11.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/virginiakm1988/ML2022-Spring/main/HW03/food11.zip [following]
--2022-03-24 08:14:32--  https://media.githubusercontent.com/media/virginiakm1988/ML2022-Spring/main/HW03/food11.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185

In [2]:
%%capture
! unzip food11.zip

# Training

In [3]:
import numpy as np
import pandas as pd
from glob import glob
import torch
import os
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from pathlib import Path
import torch.nn as nn
from tqdm import tqdm
from typing import List
from torch.utils.tensorboard import SummaryWriter
import random

In [4]:
# make the training be deterministic
my_seed = 6666
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(my_seed)
random.seed(my_seed)
torch.manual_seed(my_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(my_seed)

## Dataset

In [5]:
N_REPEAT = 2

class FoodDataset(Dataset):

    def __init__(self, path: str, transform=None, testing=False, n_repeat=1, clean=False):
        super().__init__()
        if clean:
            self.files = [
                    f for f in sorted(glob(os.path.join(path, '*.jpg')))
                if int(
                    Path(f).stem.split('_')[0]
                ) not in [1, 10]
            ]
        else:
            self.files = sorted(glob(os.path.join(path, '*.jpg')))
        assert len(self.files) > 0
        if n_repeat > 1:
            self.files = [f for f in self.files for _ in range(n_repeat)]
        self.transform = transform
        self.testing = testing

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name = self.files[idx]
        if self.transform:
            image = self.transform(Image.open(file_name))
        else:
            image = Image.open(file_name)

        if self.testing:
            label = -1
        else:
            tags = Path(file_name).stem.split('_')
            assert len(tags) == 2
            label = int(tags[0])

        return (image, label)


class MyDataset(Dataset):

    def __init__(self, files: List[Path], n_repeat: int = 0):
        super().__init__()
        assert len(files) > 0
        self.files = files
        if n_repeat > 0:
            self.files = self.files * n_repeat
        self.transform = None

    def __len__(self):
        return len(self.files)

    def set_transform(self, transform):
        self.transform = transform

    def __getitem__(self, idx):
        file_name = self.files[idx]
        if self.transform:
            image = self.transform(Image.open(file_name))
        else:
            image = Image.open(file_name)

        tags = Path(file_name).stem.split('_')
        assert len(tags) == 2
        label = int(tags[0])

        return (image, label)

In [6]:
def random_split(ratio=0.8):
    files1 = list(Path('./food11/training').glob('*.jpg'))
    files2 = list(Path('./food11/validation').glob('*.jpg'))
    files = files1+files2
    random.shuffle(files)
    n_train = int(len(files) * ratio)
    return {
        'train': MyDataset(files[:n_train], n_repeat=N_REPEAT),
        'valid': MyDataset(files[n_train:])
    }

## Transform

In [7]:
tf = {
    'train': transforms.Compose([
        #  transforms.RandomCrop(
        #      (512, 512),
        #      pad_if_needed=True
        #  ),
        #  transforms.RandomCrop(
        #      (640, 640),
        #      pad_if_needed=True
        #  ),
        #  transforms.CenterCrop((640, 640)),
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(
            degrees=(-20, 20),
            translate=(0.1, 0.3),
            scale=(0.5, 0.75)
        ),
        #  transforms.RandomRotation(degrees=(-10, 10)),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.2,
            saturation=0,
            hue=0
        ),
        transforms.ToTensor(),
        transforms.RandomErasing(),
    ]),
    'valid': transforms.Compose([
        #  transforms.CenterCrop((512, 512)),
        #  transforms.CenterCrop((640, 640)),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ]),
    'aug': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(
            degrees=(-20, 20),
            translate=(0.1, 0.3),
            scale=(0.5, 0.75)
        ),
        transforms.ColorJitter(
            brightness=0.1,
            contrast=0.2,
            saturation=0,
            hue=0
        ),
    ]),
}

In [8]:
datasets = random_split(ratio=0.7)
for key in datasets:
    datasets[key].set_transform(tf[key])

## Model

In [9]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )


    def forward(self, x):
        return self.model(x)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class_sample_count = [
    1356,
    573,
    2000,
    1313,
    1174,
    1774,
    587,
    376,
    1202,
    2000,
    941
]
loss_weight = (1 / torch.Tensor(class_sample_count)).to(device)

loaders = {
    key: DataLoader(
        datasets[key],
        batch_size=128,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    ) for key in datasets
}

CHECKPOINT = 'best.ckpt'
#  CHECKPOINT = 'clean.ckpt'
model = Classifier().to(device)
# model.load_state_dict(torch.load(CHECKPOINT))
criterion = nn.CrossEntropyLoss(weight=loss_weight)

def accuracy(logits, label):
    return (logits.argmax(dim=-1) == label).float().mean()

optimizer = torch.optim.Adam(
    model.parameters(),
    #  lr=0.0005,
    # lr=0.00001,
    #  lr=0.0001,
    lr=0.001,
    weight_decay=1e-5
)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 25, 40, 55, 70, 85], gamma=0.5)

## Main

In [ ]:
N_EPOCHS = 300
n_train = len(loaders['train'])
n_valid = len(loaders['valid'])
valid_freq = 1

best_accu = 0.
stagnation = 0
PATIENCE = 20
N_TEST_TIME_AUG = 5

writer = SummaryWriter()

for epoch in range(N_EPOCHS):
    print()
    print('Epoch:', epoch)

    # training
    model.train()
    running_accu = 0.
    running_loss = 0.
    train_bar = tqdm(loaders['train'])
    for (image, label) in train_bar:
        image = image.to(device)
        label = label.to(device)
        logits = model(image)
        loss = criterion(logits, label)
        accu = accuracy(logits, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss = loss.item()
        running_accu += accu
        running_loss += loss
        train_bar.set_description('[train] loss: %.3f, accu: %.3f' % (loss, accu))
    running_accu /= n_train
    running_loss /= n_train
    scheduler.step()
    print('[train] avg loss: %.3f, avg accu: %.3f, lr: %.5f' % (running_loss, running_accu, optimizer.param_groups[0]['lr']))
    writer.add_scalar('train/loss', running_loss, epoch)
    writer.add_scalar('train/accu', running_accu, epoch)

    if epoch % valid_freq == 0 and epoch > 15:
        # validation
        model.eval()
        running_accu = 0.
        running_loss = 0.
        valid_bar = tqdm(loaders['valid'])
        for (image, label) in valid_bar:
            image = image.to(device)
            label = label.to(device)
            with torch.no_grad():
                logits = model(image)
                if N_TEST_TIME_AUG > 0:
                    logits = nn.functional.softmax(logits, dim=1)
                    for _ in range(N_TEST_TIME_AUG):
                        logits += nn.functional.softmax(model(tf['aug'](image)), dim=1)
            accu = accuracy(logits, label).cpu()
            loss = criterion(logits, label).item()
            running_accu += accu
            running_loss += loss
            valid_bar.set_description('[valid] loss: %.3f, accu: %.3f' % (loss, accu))
        running_accu /= n_valid
        running_loss /= n_valid

        print('[valid] avg loss: %.3f, avg accu: %.3f' % (running_loss, running_accu))
        writer.add_scalar('valid/loss', running_loss, epoch)
        writer.add_scalar('valid/accu', running_accu, epoch)

        if running_accu > best_accu:
            torch.save(model.state_dict(), CHECKPOINT)
            best_accu = running_accu
            writer.add_scalar('best_accu', best_accu, epoch)
            stagnation = 0
        else:
            stagnation += 1
            if stagnation > PATIENCE:
                print('Early stopped')
                break


Epoch: 0


[train] loss: 2.211, accu: 0.148: 100%|██████████| 146/146 [04:33<00:00,  1.87s/it]


[train] avg loss: 2.397, avg accu: 0.135, lr: 0.00100

Epoch: 1


[train] loss: 2.427, accu: 0.130: 100%|██████████| 146/146 [04:38<00:00,  1.91s/it]


[train] avg loss: 2.211, avg accu: 0.177, lr: 0.00100

Epoch: 2


...

[train] loss: 0.494, accu: 0.815: 100%|██████████| 146/146 [04:39<00:00,  1.92s/it]


[train] avg loss: 0.594, avg accu: 0.761, lr: 0.00013


[valid] loss: 1.105, accu: 0.714: 100%|██████████| 32/32 [01:16<00:00,  2.39s/it]


[valid] avg loss: 1.114, avg accu: 0.708

Epoch: 41


[train] loss: 0.609, accu: 0.789:  82%|████████▏ | 120/146 [03:51<00:50,  1.94s/it]

In [ ]:
test_set = FoodDataset(
    './food11/test',
    transform=tf['valid'],
    testing=True,
)
test_loader = DataLoader(
    test_set,
    batch_size=128,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [ ]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(CHECKPOINT))
model_best.eval()
prediction = []
with torch.no_grad():
    for (image, _) in test_loader:
        image = image.to(device)
        logits = model_best(image)
        if N_TEST_TIME_AUG > 0:
            logits = nn.functional.softmax(logits, dim=1)
            for _ in range(N_TEST_TIME_AUG):
                logits += nn.functional.softmax(model_best(tf['aug'](image)), dim=1)
        pred = np.argmax(
            logits.cpu().data.numpy(),
            axis=1
        )
        prediction += pred.squeeze().tolist()

def pad4(i):
    return '0' * (4 - len(str(i))) + str(i)

df = pd.DataFrame()
df['Id'] = [pad4(i) for i in range(1, len(test_set) + 1)]
df['Category'] = prediction
df.to_csv('submission.csv',index = False)